In [2]:
from qiskit import QuantumCircuit, assemble, BasicAer, execute, QuantumRegister, ClassicalRegister, Aer
from qiskit.visualization import plot_histogram, plot_bloch_vector
from math import sqrt, pi
from qiskit.quantum_info import Statevector
from qiskit.circuit.library import QFT
from qiskit.providers.aer import QasmSimulator
from qiskit.visualization import plot_histogram
import pylatexenc
import numpy as np
import cv2
import math
import matplotlib.pyplot as plt
from matplotlib import style
import random
from image_slicer import slice
style.use('bmh')

In [3]:
n = 6
total_n = n + 1
size = int(math.sqrt(2**n))

In [4]:
def plot_image(img, title: str):
    plt.title(title)
    plt.xticks(range(img.shape[0]))
    plt.yticks(range(img.shape[1]))
    plt.imshow(img, extent=[0, img.shape[0], img.shape[1], 0], cmap='viridis')
    plt.show()

In [18]:
def normalise(img):
    amp = img.reshape(size**2, 1)
    norm = np.linalg.norm(amp)
    if(norm == 0):
        amp[0] = 1
        norm = np.linalg.norm(amp)
    amp = amp/norm
    return amp.tolist()

In [12]:
def edge_detection(img):
    D2n_1 = np.roll(np.identity(2**total_n), 1, axis=1)

    arr_h = normalise(img)
    arr_v = normalise(img.T)

    arr_h = Statevector(arr_h) 
    arr_v = Statevector(arr_v)

    qc_h = QuantumCircuit(total_n)
    qc_h.initialize(arr_h, range(1, total_n))
    qc_h.h(0)
    qc_h.unitary(D2n_1, range(total_n))
    qc_h.h(0)

    qc_v = QuantumCircuit(total_n)
    qc_v.initialize(arr_v, range(1, total_n))
    qc_v.h(0)
    qc_v.unitary(D2n_1, range(total_n))
    qc_v.h(0)

    circ_list = [qc_h, qc_v]
    back = Aer.get_backend('statevector_simulator')
    results = execute(circ_list, backend=back).result()
    sv_h = results.get_statevector(qc_h)
    sv_v = results.get_statevector(qc_v)
    
    return sv_h, sv_v

    #threshold = lambda amp: (amp > 1e-15 or amp < -1e-15)

    #edge_scan_h = np.abs(np.array([1 if threshold(sv_h[2*i+1].real) else 0 for i in range(2**n)])).reshape(size, size)
    #edge_scan_v = np.abs(np.array([1 if threshold(sv_v[2*i+1].real) else 0 for i in range(2**n)])).reshape(size, size).T

    #plot_image(edge_scan_h, 'Horizontal scan output')
    #plot_image(edge_scan_v, 'Vertical scan output')

    #edge_scan = edge_scan_h | edge_scan_v
    #plot_image(img, 'Original image')
    #plot_image(edge_scan, 'Edge Detected image')
    #return edge_scan

In [13]:
def get_edge_scan(sv_h, sv_v):
    threshold = lambda amp: (amp > 1e-15 or amp < -1e-15)

    edge_scan_h = np.abs(np.array([1 if threshold(sv_h[2*i+1].real) else 0 for i in range(2**n)])).reshape(size, size)
    edge_scan_v = np.abs(np.array([1 if threshold(sv_v[2*i+1].real) else 0 for i in range(2**n)])).reshape(size, size).T

    #plot_image(edge_scan_h, 'Horizontal scan output')
    #plot_image(edge_scan_v, 'Vertical scan output')

    edge_scan = edge_scan_h | edge_scan_v
    #plot_image(img, 'Original image')
    #plot_image(edge_scan, 'Edge Detected image')
    return edge_scan

In [14]:
def split_image(img):
    tiles = []
    for r in range(0,img.shape[0],size):
        for c in range(0,img.shape[1],size):
            tiles.append(img[r:r+size, c:c+size])
    tiles = np.array(tiles)
    tiles = tiles.reshape(int(N/size), int(N/size), size, size)
    return tiles

In [15]:
def combine_images(imgs):
    res = []
    temp = []
    for row in imgs:
        temp.append(np.hstack(row))
    res = np.vstack(temp)
    return res

In [16]:
img = cv2.imread("op.png", cv2.IMREAD_GRAYSCALE)
N = img.shape[0]
tiles = split_image(img)
print(tiles.shape)
cv2.imwrite("op_gray.png", img)

(32, 32, 8, 8)


False

In [19]:
edge_scan = []
temp_scan = []
count = 0

for row in tiles:
    for tile in row:
        a,b = edge_detection(tile)
        temp_scan.append([a,b])
        print(count)
        count = count + 1


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [31]:
print(len(temp_scan))

1024


In [38]:
x = int(N/size)
edge_scan = []
for i in range(x):
    temp = []
    for tup in temp_scan[x*i: x*i + x]:
        temp.append(get_edge_scan(tup[0], tup[1]))
    edge_scan.append(temp)


In [39]:
print(len(edge_scan))

32


In [40]:
res = combine_images(edge_scan)
print(res.shape)
for i in range(N):
    for j in range(N):
        if(res[i, j] == 1):
            res[i, j] = 0
        else:
            res[i, j] = 255
#plot_image(res, 'Edge Detected image')
cv2.imwrite("op_gray_edge.png", res)

(256, 256)


True

8 8
